CONNECTION TO DB

In [1]:
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
conn = None  # Global connection variable store


In [2]:
def connect_db():
    global conn
    if conn is not None:
        # Check if connection is still open
        try:
            cur = conn.cursor()
            cur.execute('SELECT 1')
            cur.close()
            return conn  # Return existing valid connection
        except (psycopg2.InterfaceError, psycopg2.OperationalError):
            # Connection is closed or broken, so reset it
            conn = None

    # Create new connection if none exists or previous one failed
    conn = psycopg2.connect(
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD")
    )
    return conn

# Usage example
# Later, when you need a connection:
conn = connect_db()

Connect and display structure and row count of all tables

In [13]:
# ------------- Connect and display structure and row count of all tables

tables = [
    # "geographic_locator",
    # "basic_drugs_formulary",
    # "users",
    # "beneficiary_cost",
    "plan_info"
]

conn = connect_db()

cur = conn.cursor()

for table in tables:
    print(f"Table: {table}")
    
    # Get table structure: column names and data types
    cur.execute(f"""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{table}'
        ORDER BY ordinal_position
    """)
    columns = cur.fetchall()
    
    for col in columns:
        print(f"  {col[0]} ({col[1]})")
    
    # Get row count
    cur.execute(f"SELECT COUNT(*) FROM {table}")
    row_count = cur.fetchone()[0]
    print(f"Rows: {row_count}\n")

cur.close()
conn.close()

Table: plan_info
  contract_id (character varying)
  plan_id (character varying)
  segment_id (character varying)
  contract_name (character varying)
  plan_name (character varying)
  formulary_id (character varying)
  premium (numeric)
  deductible (integer)
  ma_region_code (character varying)
  pdp_region_code (character varying)
  state (character)
  county_code (character varying)
  snp (integer)
  plan_suppressed_yn (character)
Rows: 115379



VIEW DATABASE VERSION

In [12]:
conn = connect_db()

# Create a cursor to perform database operations
cur = conn.cursor()

# Example query
cur.execute("SELECT version();")
result = cur.fetchone()
print("Database version:", result)

# Clean up
cur.close()
conn.close()

Database version: ('PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit',)


Connect and display all tables

In [12]:
# ------------- Connect and display all tables
conn = connect_db()

cur = conn.cursor()

# List all tables in the 'public' schema
cur.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
""")

tables = cur.fetchall()
for table in tables:
    print(table[0])

cur.close()
conn.close()

geographic_locator
basic_drugs_formulary
users
beneficiary_cost
plan_info


MODIFYING TABLE TO INCREASE VARCHAR LENGTH

In [6]:
conn = connect_db()

# Create a cursor to perform database operations
cur = conn.cursor()



# cur.execute("""
# ALTER TABLE geographic_locator 
#   ALTER COLUMN county_code TYPE varchar(500),
#   ALTER COLUMN statename TYPE varchar(500),
#   ALTER COLUMN county TYPE varchar(500),
#   ALTER COLUMN ma_region_code TYPE varchar(500),
#   ALTER COLUMN ma_region TYPE varchar(500),
#   ALTER COLUMN pdp_region_code TYPE varchar(500),
#   ALTER COLUMN pdp_region TYPE varchar(500);
# """)

# cur.execute("""
# ALTER TABLE basic_drugs_formulary 
# ALTER COLUMN QUANTITY_LIMIT_AMOUNT TYPE DECIMAL(7,2);
# """)


conn.commit()

# Query for varchar columns and their max length
cur.execute("""
    SELECT column_name, data_type, character_maximum_length
    FROM information_schema.columns
    WHERE table_name = 'users' 
""")

for row in cur.fetchall():
    print(row)  # Each row: (column_name, data_type, character_maximum_length)

# Clean up
# cur.close()
conn.close()

('id', 'integer', None)
('created_at', 'timestamp without time zone', None)
('updated_at', 'timestamp without time zone', None)
('oauth_provider', 'character varying', 50)
('oauth_id', 'character varying', 200)
('profile_pic_url', 'character varying', 500)
('role', 'character varying', 50)
('name', 'character varying', 100)
('email', 'character varying', 150)


DELETING A TABLE

In [8]:
conn = connect_db()

cur = conn.cursor()

# Truncate the table
cur.execute("DROP TABLE drugs_formulary_details CASCADE;")

# Commit your changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

TRUNCATE THE TABLE

In [ ]:
conn = connect_db()
cursor = conn.cursor()

# Truncate the table
cursor.execute("TRUNCATE TABLE formulary_info;")

# Commit your changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()